# **DÜNYA İNTİHAR İSTATİSTİĞİ VERİ ANALİZİ**
İnceleyeceğimiz veri seti 141 ülkede 1979-2015 yılları arası intihar eden kişi sayısını cinsiyete ve yaş gruplarına göre dağıtarak oluşturulmuştur.
Amacımız öncelikle bu veri setini görselleştirerek veri analizi yapmak ve makine öğrenmesi algoritmalarını kullanarak gelecek yıllardaki intihar sayısının tahminidir.

> # **VERİ ÖN İŞLEME**

In [ ]:
# Öncelikle kütüphanelerimizi çağırıyoruz.
import pandas as pd # Verimizi okumak ve gruplandırmak için: pandas
import numpy as np  # Verimiz üzerinde sayısal işlemler yapmak için: numpy
# Verimizi görselleştirmek için: matplot ve seaborn
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
# Kullanacağımız verimizi okuyoruz.
df = pd.read_csv("../input/who_suicide_statistics.csv")

In [ ]:
# Veri setimizin ilk 10 değerini inceliyoruz
df.head(10)

In [ ]:
# Veri setimizin son 5 değerini inceliyoruz
df.tail()

In [ ]:
# Veri setimizin satır sayısına ve öz nitelik adedine bakıyoruz.
df.shape

In [ ]:
# Kolonlarımızı ve veri tiplerimizi gözlemliyoruz.
df.info()

In [ ]:
# Sayısal değerlerimize bakıyoruz.
df.describe() 

In [ ]:
# Verimizin korelasyon durumunu gözlemliyoruz. 
sns.heatmap(df.corr())

**Verimizin korelasyon durumundan şu an ki haliyle fikir edinemiyoruz. Çünkü sayısal kolonlar iki adet ve net bir bilgi döndürmemekte.**


In [ ]:
# Verimizdeki toplam boş değer sayısını kontrol ediyoruz.
df.isnull().sum()

Veri setimizi incelediğimizde kayıp verilerimizin belirli ülkelere, belirli yıllara göre dağılımlı olduğunu görüyoruz.
<br>Örneğin Bolivya ya da Tunus ülkesine ait hiç bir popülasyon verisi mevcut değil, haliyle bu verileri elden girmek *(popülasyonun ortalama verisini ya da mod verisini girmek)* yanıltıcı olacaktır.
<br>Aynı şekilde intihar sayısı niteliği çoğunlukla yıllara göre gruplanmış durumda. Örneğin Uruguay'ın 1979 yılına, Ukrayna'nın 1983 yılına, Arnavutluk'un 1985-86, 1990-91 ve 2011-15 yılları arasına ait intihar verileri kayıp durumda. Bundan ötürü diğer yıllara ait intihar verilerini girmek de veri bütünlüğünü bozacaktır. 
 <br>**Sonuç itibariyle kayıp verileri toptan temizlemek toplam veri adedine kıyasla çok büyük bir kayıp olmayacağından en mantıklı çözüm olacaktır.**

Şu anlık yapacağımız görselleştirmelerde popülasyon kolonunu dahil etmeyeceğiz ve popülasyonun kayıp veri sayısı 5460 iken intihar sayısı kolonunun kayıp veri sayısı 2256.
<br>Haliyle öncelikle popülasyon kolonunu çıkartıp daha sonra kayıp veri temizliği yaparsak çok daha az verimiz silinecektir.


In [ ]:
# Popülasyon kolonumuzu kaldırıyoruz. Gerektiği görselleştirme kısımlarında geri çağıracağız.
df.drop(["population"], axis=1, inplace=True)

In [ ]:
# Herhangi bir özniteliği boş olan verileri temizliyoruz
df = df.dropna(axis=0, how="any")

In [ ]:
# Boş değerlerimizin kalıp kalmadığını gözlemleyelim
df.isnull().sum()

> # **VERİ GÖRSELLEŞTİRME**

* **Yıllara göre dünyadaki intihar olay sıklığını gözlemleyelim.**

In [ ]:
# Kullanacağımız kolonları ayırıyoruz.
years = df.iloc[:, 1].values.reshape(-1,1)
suicides = df.iloc[:, 4].values.reshape(-1,1)

In [ ]:
# Histogram (frekans grafiği) ile intihar olayının en çok tekrar ettiği yılları görebiliriz.
plt.figure(figsize=(20,10))
plt.hist(years, bins=50)
plt.xlabel("Yıllar")
plt.ylabel("İntihar Sıklığı")

**Gözle görünür oranda 2000'li yıllardan sonra intihar olaylarının sıklaştığını söyleyebiliriz.**

* **Ülkelerin toplam intihar sayılarını gözlemleyelim.**

In [ ]:
# Bütün ülkeleri intihar sayılarına göre gruplandırıyoruz ve dataframe'imizi düzenliyoruz
country_suicides = pd.DataFrame(df.groupby("country")["suicides_no"].sum())
country_suicides['country'] = country_suicides.index
country_suicides = country_suicides.reset_index(drop = True)

country_suicides.head()

In [ ]:
# Ülkeleri intihar sayılarına göre düzenleyip en çok ilk 50 ülkemizi ayırıyoruz
sorted_data = country_suicides.sort_values("suicides_no", ascending=False)
sorted_data = sorted_data.iloc[:50,:]

*(Toplamda 141 ülkemiz mevcut ve bu görselleştirmek için çok fazla)*

In [ ]:
# Görselimizi oluşturuyoruz
plt.figure(figsize=(20,10))
sns.barplot(x=sorted_data.country, y=sorted_data.suicides_no)
plt.xticks(rotation= 90)
plt.xlabel('Ülkeler')
plt.ylabel('Toplam İntihar Sayısı')
plt.title('İNTİHARLAR')

**İntihar sayısının en yüksek olduğu başlıca ülkelerin Rusya, Amerika, Japonya ve Fransa gibi popülasyonun da yüksek olduğu ülkeler olduğunu gözlemliyoruz. Fakat bu veri intihar/popülasyon oranından ziyade kalabalık ülkelerin (doğal olarak) intihar sayısının da yüksek olacağından dominant bir veri dağılımına sebep olmaktadır.**
<br>
<br>**İlerleyen safhalarda intihar/popülasyon oranını da inceleyeceğiz**

* **Ülkelerin toplam intihar sayısını ve kadın erkek yüzdesini hesaplayalım**

In [ ]:
# Ülkeleri isim olarak ayırıyoruz
unique = pd.DataFrame(df.country.unique(), columns = ["countries"])
unique.sample(10)

In [ ]:
# Toplayacağımız verileri saklayacağımız veri tablosunu oluşturuyoruz.
suicides_by_gender = pd.DataFrame(columns=['country', # ülke
                                           'total s', # total suicide:  toplam intihar sayısı
                                           'per f',   # percent female: kadın intiharları yüzdesi
                                           'per m',]) # percent male:   erkek intiharları yüzdesi

Burada yukarıda ayırdığımız ülke isimlerini bir döngüde dondürecek ve ana veri setimizde eşleşen ülkelerin her intihar sayılarını kadın erkek olarak ayıracak ve yüzdelerini hesaplayacağız.


In [ ]:
for index, rows in unique.iterrows():
   test_df = df[df["country"] == rows["countries"]]
   suicides_by_gender = suicides_by_gender.append({'country': rows["countries"],
        'total s' : df[df["country"] == rows["countries"]].suicides_no.sum(),
        'per f': test_df[test_df["sex"] == "female"].suicides_no.sum()/df[df["country"] == rows["countries"]].suicides_no.sum()*100,
        'per m': test_df[test_df["sex"] == "male"].suicides_no.sum()/df[df["country"] == rows["countries"]].suicides_no.sum()*100}, ignore_index=True)

In [ ]:
# Verimizdeki intihar sayısı baz alınarak aykırı verileri temizliyoruz.
P = np.percentile(suicides_by_gender['total s'], [10, 100])
suicides_by_gender = suicides_by_gender[(suicides_by_gender['total s'] > P[0]) & (suicides_by_gender['total s'] <= P[1])]

In [ ]:
# Oluşan veri tablomuzu inceleyelim
suicides_by_gender.head()

In [ ]:
# Verimizi kadın intiharları yüzdesine göre sıralayalım.
suicides_by_gender = suicides_by_gender.sort_values("per f", ascending=False)
suicides_by_gender.head(10)

In [ ]:
# Verimizi erkek intihar yüzdesine göre sıralayalım.
suicides_by_gender = suicides_by_gender.sort_values("per m", ascending=False)
suicides_by_gender.head(10)

* **Ülkelerin intihar sayılarını yaş aralıklarına göre gruplandıralım**

In [ ]:
# Ülke isimlerini "unique" içerisinde daha önceden kaydetmiştik.
# Toplayacağımız verileri saklayacağımız veri tablosunu oluşturuyoruz.
suicides_by_age = pd.DataFrame(columns=[
    'country',   # ülke
    'total s',   # total suicide: toplam intihar sayısı
    'per 5-14',  # 5 ve 14 yaş arası kadın ve erkeklerin toplam yüzdesi
    'per 15-24', # 15 ve 24 yaş arası kadın ve erkeklerin toplam yüzdesi
    'per 25-34', # 25 ve 34 yaş arası kadın ve erkeklerin toplam yüzdesi
    'per 35-54', # 35 ve 54 yaş arası kadın ve erkeklerin toplam yüzdesi
    'per 55-74', # 55 ve 74 yaş arası kadın ve erkeklerin toplam yüzdesi
    'per 75+'])  # 75 yaş ve üzeri kadın ve erkeklerin toplam yüzdesi

In [ ]:
# Döngümüzle ana veri tablomuzdan eşleşen ülkelerin yaş aralıklarındaki intiharlar düzenleniyor.
for index, rows in unique.iterrows():
   test_df = df[df["country"] == rows["countries"]]
   s = df[df["country"] == rows["countries"]].suicides_no.sum()
   suicides_by_age = suicides_by_age.append({'country': rows["countries"],
        'total s': df[df["country"] == rows["countries"]].suicides_no.sum(),
        'per 5-14': test_df[test_df["age"] == "5-14 years"].suicides_no.sum()/s *100, 
        'per 15-24': test_df[test_df["age"] == "15-24 years"].suicides_no.sum()/s*100, 
        'per 25-34': test_df[test_df["age"] == "25-34 years"].suicides_no.sum()/s*100, 
        'per 35-54': test_df[test_df["age"] == "35-54 years"].suicides_no.sum()/s*100, 
        'per 55-74': test_df[test_df["age"] == "55-74 years"].suicides_no.sum()/s*100,
        'per 75+': test_df[test_df["age"] == "75+ years"].suicides_no.sum()/s*100}, ignore_index=True)

In [ ]:
# Verimizdeki intihar sayısını göz önünde bulundurarak aykırı verileri temizliyoruz.
P = np.percentile(suicides_by_age['total s'], [10, 100])
suicides_by_age = suicides_by_age[(suicides_by_age['total s'] > P[0]) & (suicides_by_age['total s'] <= P[1])]

In [ ]:
# Oluşan veri tablomuzu inceleyelim
suicides_by_age.head()

In [ ]:
# Verimizi 5-14 yüzdesine göre sıralayalım.
suicides_by_age = suicides_by_age.sort_values("per 5-14", ascending=False)
suicides_by_age.head(10)

In [ ]:
# Verimizi 15-24 yüzdesine göre sıralayalım.
suicides_by_age = suicides_by_age.sort_values("per 15-24", ascending=False)
suicides_by_age.head(10)


In [ ]:
# Verimizi 25-34 yüzdesine göre sıralayalım.
suicides_by_age = suicides_by_age.sort_values("per 25-34", ascending=False)
suicides_by_age.head(10)

In [ ]:
# Verimizi 35-54 yüzdesine göre sıralayalım.
suicides_by_age = suicides_by_age.sort_values("per 35-54", ascending=False)
suicides_by_age.head(10)

In [ ]:
# Verimizi 55-74 yüzdesine göre sıralayalım.
suicides_by_age = suicides_by_age.sort_values("per 55-74", ascending=False)
suicides_by_age.head(10)

In [ ]:
# Verimizi 75+ yüzdesine göre sıralayalım.
suicides_by_age = suicides_by_age.sort_values("per 75+", ascending=False)
suicides_by_age.head(10)

*  **Ülkelerin intihar / popülasyon oranlarını hesaplayalım.**
<br>
 <br>**Bunun için ülkelerin toplam intihar sayısını popülasyonlarının ortalamasına bölüp yüzdesini hesaplayacağız.**
 <br>(*Önceki verimizde veri kaybının yüksek olmaması için popülasyon öz niteliğini çıkartmıştık. Tekrar çağırıp eksik verilerimizi temizliyoruz.*)


In [ ]:
df2 = pd.read_csv("../input/who_suicide_statistics.csv")
df2 = df2.dropna(axis=0, how="any")
df2.isnull().sum()

***Ülke isimlerini "unique" içerisinde daha önceden kaydetmiştik fakat veri setimizden önceki veri setine kıyasla daha farklı ve daha fazla veri sildik. Bundan ötürü tekrar ülke isimlerini çekmemiz gerek***

In [ ]:
unique = pd.DataFrame(df2.country.unique(), columns = ["countries"])
unique.sample(10)

In [ ]:
# Toplayacağımız verileri saklayacağımız veri tablosunu oluşturuyoruz.
suicides_by_pop = pd.DataFrame(columns=['country', 
                                           'total s', 
                                           'mean pop',
                                           'per s'])

In [ ]:
# Her bir ülkeyi gezip intiharların toplamını popülasyonun ortalamasını alıyor, oluşturduğumuz dataframe'e ekliyoruz.
for index, rows in unique.iterrows():
   test_df = df2[df2["country"] == rows["countries"]]
   suicides_by_pop = suicides_by_pop.append({
        'country': rows["countries"],
        'total s' : test_df.suicides_no.sum(),
        'mean pop': test_df.population.mean(),
        'per s': test_df.suicides_no.sum()/test_df.population.mean()*100}, 
            ignore_index=True)

In [ ]:
# Aykırı verileri temizliyoruz.
P = np.percentile(suicides_by_pop['per s'], [10, 100])
suicides_by_pop = suicides_by_pop[(suicides_by_pop['per s'] > P[0]) & (suicides_by_pop['per s'] <= P[1])]

In [ ]:
# Oluşan veri tablomuzu inceleyelim
suicides_by_pop.head()

In [ ]:
# Verimizi intihar/popülasyon oran yüzdesine göre sıralayalım.
suicides_by_pop = suicides_by_pop.sort_values("per s", ascending=False)
suicides_by_pop.head(10)

> # **VERİ MODELİ İŞLEME**

Bu bölümümüzde regresyon modelleri oluşturarak veri setimizde eğitimler uygulayacak ve belirlediğimiz öz niteliklere göre en iyi sonucu veren regresyon modelini saptayacağız.

Deneyeceğimiz modellerin yıllara göre intihar sayısını tahmin etmesini bekleyeceğiz.

* Linear Regression
* Support Vector Regression
* Desicion Tree Regression
* Random Forest Regression
* K-Nearest Neighbor Regression

In [ ]:
# Regresyon modellerimizin başarısını ölçmek için r2 score fonksiyonunu kullanacağız.
# Değer ne kadar 1'e yakınsa o kadar iyi sonuç alınmıştır demektir.
from sklearn.metrics import r2_score

**Eğitim ve test verisinde kullanacağımız verilerimizi hazırlıyoruz.**

In [ ]:
# Yıllara göre intihar sayılarının toplamını grupluyoruz.
analyze_df = pd.DataFrame(df.groupby("year")["suicides_no"].sum())

In [ ]:
analyze_df.head()

In [ ]:
analyze_df.tail()

**Verileri incelediğimizde 2016 yılının verisi hatalı olduğu için o yılı elimine ediyoruz.**

In [ ]:
# Girdi değerlerine yılları, çıktı değerlerine intiharları atıyoruz. 
x = pd.DataFrame(analyze_df.index)
y = analyze_df.iloc[:, 0]

# 2016 yılını ve intihar değerini temizliyoruz.
x = x.iloc[:-1, :].values.reshape(-1,1)
y = analyze_df.iloc[:-1, 0].values.reshape(-1,1)

### Regresyon modellerini oluşturmaya hazırız.

 İlk olarak **Polynomial Linear Regression** modelini uygulayacağız.
 <br>Bu modelde oluşturulacak fonksiyonu görselleştirmek için datayı train-test olarak bölmeyeceğiz.
<br>** (Böldüğümüz taktirde fonksiyon eğrisini çizemeyecektir.)**

>  ## **LINEAR REGRESSION**

In [ ]:
# Gerekli kütüphaneleri çağırıyoruz.
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# Modellerimizi oluşturuyoruz.
lr = LinearRegression()

# 4. dereceden bir fonksiyon kullanacağız.
pf = PolynomialFeatures(degree=4)

In [ ]:
# Öncelikle girdi değerlerimizi polinomal forma çeviriyoruz. Çünkü polinomal bir fonksiyon oluşturduysak o dereceden girdiler sağlamalıyız.
x_pol = pf.fit_transform(x)

In [ ]:
# Modelimizi eğitiyoruz
lr.fit(x_pol, y)

In [ ]:
# Modelimizi eğittik, şimdi ise test edeceğiz. Öncelikle test datamızı polinomal hale getiriyoruz.
x_pol2 = pf.fit_transform(x)

In [ ]:
# Çevirdiğimiz test verisini modele gönderip tahmin edilen değerlerimizi göreceğiz.
y_pred = lr.predict(x_pol2)

In [ ]:
# Orijinal değerler ile fonksiyonumuzun türettiği verileri kıyaslamak için görselleştirelim.
plt.figure(figsize=(20,10))
plt.scatter(x, y)
plt.plot(x, y_pred, color="red")
plt.xlabel("Yıllar")
plt.ylabel("İntihar Sayıları")

In [ ]:
# Regresyon score değerini gözlemleyelim
r2_score(y, y_pred)

**Uygulayacağımız diğer modellerde datamızı train test olarak bölebiliriz.**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

Kullanacağımız diğer regresyon modellerinde girdi ve çıktı değerlerinin normalize edilmesi gerekmektedir. Bunun nedeni ise girdi ve çıktı değerlerinin sayısal farkının çok yüksek olma durumunda algoritmanın yanılmasına sebep olmasını önlemektir.
<br>**Örneğin;** KNN modeli herhangi bir girdi değerinin sonucunu onun komşuluklarını baz alarak belirler. Komşuluk hesaplamalarında ise Pisagor denklemini kullanır.
<br>Karesi alınacak kenarların herhangi birinin değeri 0-1 arasında bir değer ise *(örneğin x = 0.1, 0.3, 0.05 gibi değerlere sahipse)*, bu sayı karesi alındıkça küçülecek, diğer kenar 1'den büyükse de *(örneğin y = 1000, 2200, 50 000 gibi değerlere sahipse)*, karesi alındıkça büyüyecektir ve 1'den büyük kenar formül hesaplanmasında baskınlık yaratacak dolayısıyla da algoritmanın yanlış çalışmasına sebep olacaktır. Böyle bir durumu önlemek için değerlere normalizasyon işlemi uygulanır.

In [ ]:
# Kullanacağımız modellerde girdi değerlerimize normalizasyon uygulamalıyız. Bunun için gerekli kütüphaneleri çağırıyoruz.
from sklearn.preprocessing import StandardScaler
# Girdi çıktı değerleri için ayrı birer scale modeli üretmeliyiz, çağırılan her model ancak normalize ettiği veri ölçütünü de-normalize edebilir.
sc_x = StandardScaler()
sc_y = StandardScaler()
x_train = sc_x.fit_transform(x_train)
y_train = sc_y.fit_transform(y_train)

# Test edeceğimiz girdi değerlerini de normalize etmeliyiz.
x_test = sc_x.transform(x_test)

### Eğitim ve test değerlerimiz hazır.

>  ## **SUPPORT VECTOR REGRESSION**

In [ ]:
# SVR için gerekli kütüphanemizi çağırıyoruz
from sklearn.svm import SVR

**SVR modeli için bir çok kernel seçeneğimiz mevcut. Sırasıyla 'rbf', 'linear', 'poly' ve 'sigmoid' kernellerini deneyip, r2_score değerlerini gözlemleyeceğiz.
**

In [ ]:
# rbf
model = SVR(kernel="rbf")
model.fit(x_train, y_train)
y_pred_rbf = model.predict(x_test)
# Normalize ederek eğittiğimiz veriler bize normalize edilmiş bir halde dönecektir. 
# Gerçek değerlerini görmek için de-normalize (inverse transform) etmemiz gerekmekte.
y_pred_rbf = sc_y.inverse_transform(y_pred_rbf)

In [ ]:
# linear
model = SVR(kernel="linear")
model.fit(x_train, y_train)
y_pred_linear =  sc_y.inverse_transform(model.predict(x_test))

In [ ]:
# poly
model = SVR(kernel="poly")
model.fit(x_train, y_train)
y_pred_poly =  sc_y.inverse_transform(model.predict(x_test))

In [ ]:
# sigmoid
model = SVR(kernel="sigmoid")
model.fit(x_train, y_train)
y_pred_sgm =  sc_y.inverse_transform(model.predict(x_test))

**Çıktı değerlerini gözlemliyoruz.**

In [ ]:
print(r2_score(y_test, y_pred_rbf))
print(r2_score(y_test, y_pred_linear))
print(r2_score(y_test, y_pred_poly)) 
print(r2_score(y_test, y_pred_sgm)) 

**En yüksek perfonması gösteren kernel RBF oluyor.**
<br>**0.8887428684867584**


>  ## **DESICION TREE REGRESSION**

In [ ]:
from sklearn.tree import DecisionTreeRegressor
# Ağacımızın derinliğini 100 birim belirledik.
model = DecisionTreeRegressor(max_depth = 100, random_state=0)
model.fit(x_train, y_train)
y_pred = sc_y.inverse_transform(model.predict(x_test))

In [ ]:
# Model score'umuzu kontrol ediyoruz.
r2_score(y_test, y_pred)

**Fakat veri setimiz çok büyük değil ve 100'den çok daha az bir değer de belki çok daha fazla perfonmans gösterebilir.
<br>Hangi değerin daha uygun olduğunu bulmak için farklı bir yöntem deneyelim.**
<br><br>**Döngümüz ağaç derinliğimizi 1'den 15'e kadar indirecek ve r2 score değeri score listemize eklenecek, böylelikle en uygun değeri saptayabileceğiz.**


In [ ]:
score=[]
for i in range(1, 15):
    model = DecisionTreeRegressor(max_depth = i, random_state=0)
    model.fit(x_train, y_train)
    y_pred = sc_y.inverse_transform(model.predict(x_test))
    score.append(r2_score(y_test, y_pred))

In [ ]:
# Score çıktılarımızı grafik üzerinde inceleyelim
plt.figure(figsize=(20,10))
plt.plot(range(1,15), score)
plt.xlabel("Derinlik")
plt.ylabel("R2 Score")

**Tablomuzdan da görüleceği üzere en uygun derinlik seviyesi 4 ve r2_score değeri;<br>0.9200627081915808**

>  ## **RANDOM FOREST REGRESSION**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Desicion Tree yöntemimizde olduğu gibi en uygun dallanma sayısını görmek için döngü kuralım.
score = []
for i in range(1, 50):
    model = RandomForestRegressor(n_estimators = i, random_state = 0)
    model.fit(x_train, y_train)    
    y_pred = sc_y.inverse_transform(model.predict(x_test))    
    score.append(r2_score(y_test, y_pred))

In [ ]:
# Score çıktılarımızı grafik üzerinde inceleyelim
plt.figure(figsize=(20,10))
plt.plot(range(1, 50), score)
plt.xlabel("Dallanma Sayısı")
plt.ylabel("R2 Score")
plt.show()

**Grafiğimizden de görüleceği üzere en uygun dallanma seviyesi 12 ve r2_score değeri; <br> 0.9238047137712131**


>  ## **K-NEAREST NEIGHBOR REGRESSION**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Komşu sayımızı bir döngü ile belirleyip score değerini grafik üzerinde gözlemleyebiliriz
score = []
for i in range(1, 27):
    model = KNeighborsRegressor(n_neighbors = i) # n_neighbors = k
    model.fit(x_train,y_train)
    y_pred = sc_y.inverse_transform(model.predict(x_test))    
    score.append(r2_score(y_test, y_pred))

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(range(1, 27), score)
plt.xlabel("Komşuluk Sayısı")
plt.ylabel("R2 Score")

**Sonuçlarımıza baktığımızda en uygun komşuluk değerinin 2 olduğunu ve r2_score değerinin; <br>0.9327148635745971 olduğunu görüyoruz.**


**Elimizdekilere göre incelediğimiz veri setine en uygun regresyon modelimiz 2 komşuluk değeri ile K-Nearest Neighbor regresyon modeli oluyor.**

*  Linear Reg. : 0.8467048048511636 
*  Support Vector Reg.: 0.8887428684867584
*  Desicion Tree Reg.: 0.9200627081915808
*  Random Forest Reg.: 0.9238047137712131
*  K-Nearest Neighbor Reg.: **0.9327148635745971**